# Notebook zur LR-Zerlegung

**Abgabe in den Programmiertutorien am 6. und 7. Februar 2025. Falls Sie Unterstützung bei der Bearbeitung der Programmieraufgabe brauchen, wenden Sie sich frühzeitig an Ihren Tutor oder melden Sie sich im Forum.**

Benötigte Module für dieses Notebook:

In [1]:
import numpy as np

Sei $A\in\mathbb{R}^{n\times n}$ eine reguläre Matrix. Nach Satz 4.1 aus dem Skript können wir mit Hilfe des Gauß-Algorithmus eine LR-Zerlegung durchführen:
\begin{align*}
        &PA = LR
\end{align*}
mit einer Permutationsmatrix $P$ und
\begin{align*} \\
        &L = \begin{bmatrix}
1 & 0      & 0      & \cdots & 0 \\
\ell_{2,1} & 1 & 0      & \cdots & 0 \\
\ell_{3,1} & \ell_{3,2} & 1 & \cdots & 0 \\
\vdots & \vdots & \vdots & \ddots & 0 \\
\ell_{n,1} & \ell_{n,2} & \ell_{n,3} & \cdots & 1
\end{bmatrix}, \quad R = \begin{bmatrix}
r_{1,1} & r_{1,2} & r_{1,3} & \cdots & r_{1,n} \\
0      & r_{2,2} & r_{2,3} & \cdots & r_{2,n} \\
0      & 0      & r_{3,3} & \cdots & r_{3,n} \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
0      & 0      & 0      & \cdots & r_{n,n}
\end{bmatrix}, \quad r_{i,i} \neq 0 \text{ für } i=1,...,n.
\end{align*}
    Die Einträge von $ L $ sind die $ \ell_{i,j} $ aus dem Gauß-Algorithmus.




### Teil 1: LR-Zerlegung ohne Pivotisierung

Im ersten Teil der Programmieraufgabe betrachten wir die LR-Zerlegung ohne Pivotisierung:
\begin{align*}
A=LR
\end{align*}
mit den Matrizen $L$ und $R$ aus dem vorherigen Abschnitt.

**a) Schreiben Sie eine Prozedur `LR`, welche als Eingabe eine reguläre Matrix $A\in\mathbb{R}^{n\times n}$ nimmt und die LR-Zerlegung für diese Matrix $A$ durchführt. Die Prozedur soll die Matrizen $L$ und $R$ ausgeben. Falls die Prozedur während der Iteration einen Nulleintrag auf der Diagonalen von der Matrix $R$ aufweisen sollte, dann brechen Sie den Prozess ab.**

**Hinweis:** *Initialisieren Sie die Matrix $L$ mit einer Einheitsmatrix mit der gleichen Größe wie die Matrix $A$ und die Matrix $R$ mit einer Kopie von der Matrix $A$. Belegen Sie dann die Matrizen $L$ und $R$ entsprechend des Gauß-Algorithmus.*

In [2]:
def LR(A):
    n = A.shape[0]
    R = A.copy()
    L = np.eye(n)
    for j in range(n-1):
        for i in range(j+1,n):
            if R[j,j] ==0:
                print("Nullelement auf der Diagonalen")
                break
            L[i,j] = R[i,j]/R[j,j]
            for k in range(j,n):
                R[i,k] = R[i,k] - L[i,j] * R[j,k]

    return L, R

**Überprüfen Sie das Ergebnis ihrer Prozedur mit dem folgenden Codeausschnitt:**

**Hinweis:** *Der numpy Befehl `np.allclose` gibt "True" zurück, falls die Matrizen elementenweise (innerhalb einer Toleranz) gleich sind, sonst gibt der Befehl "False" zurück.*

In [3]:
A = np.array([[1,2,3,4],[4,3,2,1],[-1,1,-1,1],[-1,1,1,-1]], dtype='float64' )   
L, R = LR(A)
print('Untere Dreiecksmatrix L:')
print(L)
print('Obere Dreiecksmatrix R:')
print(R)
print('LR:')
print(L@R)
print('A:')
print(A)
print(np.allclose(A,L@R))

Untere Dreiecksmatrix L:
[[ 1.   0.   0.   0. ]
 [ 4.   1.   0.   0. ]
 [-1.  -0.6  1.   0. ]
 [-1.  -0.6  0.5  1. ]]
Obere Dreiecksmatrix R:
[[  1.   2.   3.   4.]
 [  0.  -5. -10. -15.]
 [  0.   0.  -4.  -4.]
 [  0.   0.   0.  -4.]]
LR:
[[ 1.  2.  3.  4.]
 [ 4.  3.  2.  1.]
 [-1.  1. -1.  1.]
 [-1.  1.  1. -1.]]
A:
[[ 1.  2.  3.  4.]
 [ 4.  3.  2.  1.]
 [-1.  1. -1.  1.]
 [-1.  1.  1. -1.]]
True


### Teil 2: LR-Zerlegung mit Pivotisierung

Für die Wahl der Pivotelemente gehen wir nach Kapitel 4.2 im Skript vor und wählen im $(k+1)$-ten Schritt das Pivotelement
\begin{align*}
    \vert a_{j,k+1}^{(k)}\vert = \max_{i = k+1,...,n} \vert a_{i,k+1}^{(k)}\vert.
\end{align*}


**b) Schreiben Sie eine Prozedur `LR_pivot`, welche als Eingabe eine reguläre Matrix $A\in\mathbb{R}^{n\times n}$ nimmt und die LR-Zerlegung für diese Matrix $A$ mit Spaltenpivotsuche gemäß des obigen Kriteriums durchführt. Die Prozedur soll die Matrizen $L$, $R$ und $P$ ausgeben.**

**Hinweis:** *Initialisieren Sie auch $P$ als Einheitsmatrix mit der selben Größe wie $A$. Falls das Diagonalelement nicht das Pivotelement ist, dann müssen Zeilen innerhalb der Matrizen $L$, $R$ und $P$ vertauscht werden. Achten Sie darauf, dass bei der Matrix $L$ nicht die komplette Zeile angepasst wird, da die Einsen auf der Diagonalen erhalten bleiben müssen.* 

In [4]:
def LR_pivot(A):
    n = A.shape[0]
    R = A.copy()
    L = np.eye(n)
    P = np.eye(n)

    for j in range(n - 1):
        # Finde betragsmäßig größtes Element in der j-ten Spalte
        index = j + np.argmax(np.abs(R[j:, j]))
        
        # Zeilentausch in den Matrizen L, R und P
        if index != j:  # Tausche Zeile j und index nur dann, falls das betragsmäßig größte Element nicht an der j-Stelle ist
            R[[j, index], :] = R[[index, j], :]
            P[[j, index], :] = P[[index, j], :]
            if j > 0:  # Tausche Zeile j und index, aber nur bis zum j-ten Spalteneintrag
                L[[j, index], :j] = L[[index, j], :j]
        
        # L und R updaten
        for i in range(j + 1, n):
            L[i, j] = R[i, j] / R[j, j]
            R[i, j:] -= L[i, j] * R[j, j:]

    return L, R, P

In [5]:
A = np.array([[1,2,3,4],[4,3,2,1],[-1,1,-1,1],[-1,1,1,-1]], dtype='float64' )   
L, R, P = LR_pivot(A)
print(A)
print('Permutationsmatrix P:')
print(P)
print('Untere Dreiecksmatrix L:')
print(L)
print('Obere Dreiecksmatrix R:')
print(R)
print('PA:')
print(P@A)
print('LR:')
print(L@R)
print(np.allclose(P@A,L@R))

[[ 1.  2.  3.  4.]
 [ 4.  3.  2.  1.]
 [-1.  1. -1.  1.]
 [-1.  1.  1. -1.]]
Permutationsmatrix P:
[[0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]]
Untere Dreiecksmatrix L:
[[ 1.          0.          0.          0.        ]
 [-0.25        1.          0.          0.        ]
 [ 0.25        0.71428571  1.          0.        ]
 [-0.25        1.          0.7         1.        ]]
Obere Dreiecksmatrix R:
[[ 4.          3.          2.          1.        ]
 [ 0.          1.75       -0.5         1.25      ]
 [ 0.          0.          2.85714286  2.85714286]
 [ 0.          0.          0.         -4.        ]]
PA:
[[ 4.  3.  2.  1.]
 [-1.  1. -1.  1.]
 [ 1.  2.  3.  4.]
 [-1.  1.  1. -1.]]
LR:
[[ 4.  3.  2.  1.]
 [-1.  1. -1.  1.]
 [ 1.  2.  3.  4.]
 [-1.  1.  1. -1.]]
True


## Teil 3: LR-Zerlegungen im Vergleich

Wir betrachten nun die Matrix
\begin{align*}
A= \left( \begin{array}{cc}
\delta & -1\\
1&1\\
   \end{array} \right)
\end{align*}
für verschiedene $0 < \delta \ll 1$.  Die Matrix ist invertierbar und eine LR-Zerlegung ist auch ohne Zeilenvertauschungen möglich.

**(d) Führen Sie beide Prozeduren `LR` und `LR_pivot` mit der Matrix $A$ und $\delta=10^{-m}$ für $m=1,2,\ldots$ durch und beobachten Sie, ob das Produkt der Matrizen $L$ und $R$ tatsächlich der Matrix $A$ bzw. $PA$ entspricht, indem Sie den betragsmäßig größten Fehler aller Einträge ausgeben. Ab welchem Wert von $m$ treten bei der LR-Zerlegung ohne Zeilenvertauschungen Fehler auf? Erklären Sie, wie dieser Wert von $m$ zustande kommt. Erklären Sie außerdem die Größe des Fehlers und warum beide Prozeduren verschiedene Fehler liefern.**

**Hinweis:** *Nachdem Sie das $m$ bestimmt haben, ist es hilfreich die einzelnen Schritte der LR-Zerlegung mit und ohne Pivotisierung auf einem Blatt Papier aufzuschreiben, um das Ergebnis zu begründen. Überlegen Sie sich dabei, welche Zahlen für den Computer darstellbar sind.*

**Lösung:** Für $m=16$ ist der Fehler zum ersten mal unterschiedlich. Die LR Zerlegung ohne Pivotisierung liefert den Fehler 1. 

Erklärung: Die exakte LR-Zerlegung ohne Pivotisierung ist
$$ A = \begin{pmatrix} 1 & 0 \\ \frac{1}{\delta} & 1 \end{pmatrix} \begin{pmatrix} \delta & -1 \\ 0 & 1 + \frac{1}{\delta} \end{pmatrix}. $$
Die Zahl $\delta$ selbst ist auch für große Wert von $m$ im PC ohne nennenswerte Rundungsfehler darstellbar, genauso die Zahl $\frac{1}{\delta} = 10^m$. Dies gilt allerdings nicht für die Zahl $1 + \frac{1}{\delta} = 1 + 10^m = (1+10^{-m}) \cdot 10^m$. Ist $m$ zu groß, so stehen in der Mantisse nicht ausreichend Stellen zur Verfügung, um die Zahl $1+10^{-m}$ zu speichern. Stattdessen geht der Summand $10^{-m}$ einfach verloren. Nach Tutoriumsaufgabe 15/Hausaufgabe 17 ist die kleinste darstellbare Zahl $>1$ durch $1+2\epsilon$ mit der Maschinengenauigkeit $\epsilon$ gegeben. In IEEE 754 double precision gilt $\epsilon \approx 1.1 \cdot 10^{-16}$. Für $\delta < \epsilon$ wird also $1+\delta$ zur Zahl $1$ gerundet, also $1 + \frac{1}{\delta}$ zur Zahl $\frac{1}{\delta}$ gerundet. Der Algorithmus liefert also die fehlerhafte LR-Zerlegung
$$ \widetilde{L} = \begin{pmatrix} 1 & 0 \\ \frac{1}{\delta} & 1 \end{pmatrix}, \qquad 
   \widetilde{R}= \begin{pmatrix} \delta & -1 \\ 0 & \frac{1}{\delta} \end{pmatrix}. $$
Für das Produkt $\widetilde{L}\widetilde{R}$ gilt dann
$$ \widetilde{L}\widetilde{R} = \begin{pmatrix} \delta & -1 \\ 1 & 0 \end{pmatrix}.$$
Im rechten unteren Eintrag ist der Fehler $1$.

Bei der LR-Zerlegung mit Spaltenpivotsuche werden zunächst die Zeilen von $A$ vertauscht. In exakter Arithmetik erhält man die Zerlegung
$$ \begin{pmatrix} 1 & 1 \\ \delta & -1 \end{pmatrix} = \underbrace{ \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix} }_{=P} A = \begin{pmatrix} 1 & 0 \\ \delta & 1 \end{pmatrix} \begin{pmatrix} 1 & 1 \\ 0 & -1-\delta \end{pmatrix}.$$
Auch hier wird $-1-\delta$ für $m>16$  im PC nur als $1$ dargestellt, man erhält die fehlerhafte Zerlegung
$$ \widetilde{L} = \begin{pmatrix} 1 & 0 \\ \delta & 1 \end{pmatrix}, \qquad 
   \widetilde{R}= \begin{pmatrix} 1 & 1 \\ 0 & -1 \end{pmatrix}. $$
Für das Produkt $\widetilde{L}\widetilde{R}$ gilt dann
$$ \widetilde{L}\widetilde{R} = \begin{pmatrix} 1 & 1 \\ \delta & -1+\delta \end{pmatrix}.$$

Man kann sich nun noch überlegen, was bei Rundung der Zahl $-1+\delta = - (1-\delta)$ ins Gleitkommasystem passiert. Die größtmögliche darstellbare Zahl $<1$ in einem Gleitkommasystem ist die Zahl $1-\epsilon$ mit der Maschinengenauigkeit $\epsilon$ (also hier ungefähr $1-1.1\cdot 10^{-16}$). Achtung: Anders als bei der kleinstmöglichen darstellbaren Zahl $>1$, nämlich $1+2\epsilon$, kommt der Faktor $2$ hier nicht vor!. Für $\delta= 10^{-16}$ wird $-1+\delta = - (1-\delta)$ also auf $- (1-\epsilon) = -1 + \epsilon$ gerundet. Der Fehler im rechten unteren Eintrag gegenüber der Matrix $PA$ ist also genau die Maschinengenauigkeit $\epsilon$, alle anderen Einträge stimmen mit denen von $PA$ überein.

In [7]:
# Definiere schlecht konditionierte Matrix
m = 16
delta = 10**(-m)
A = np.array([[delta,-1],[1 , 1]])

print("Original matrix A:")
print(A)

# LR ohne Pivotisierung
L, R = LR(A)

# LR mit Pivotisierung
L_pivot, R_pivot, P_pivot= LR_pivot(A)

# Check the factorization errors
print("\nLR Zerlegung ohne Pivotisierung (A - LR):")
print(np.max(np.abs(A - L @ R)))

print("LR Zerlegung mit Pivotisierung (PA - LR):")
print(np.max(np.abs(P_pivot@A - L_pivot @ R_pivot)))

Original matrix A:
[[ 1.e-16 -1.e+00]
 [ 1.e+00  1.e+00]]

LR Zerlegung ohne Pivotisierung (A - LR):
1.0
LR Zerlegung mit Pivotisierung (PA - LR):
1.1102230246251565e-16
